# simaple QA예제

### ElasticSearch에서 ReRank가 통합되어 있어서 <br> es에 rerank를 추가 <br>es의 kibana에서 dev tools 혹은 wget, curl을 통하 아래 커맨드를 수행

### Rerank를 지원하는 ES 버젼은 8.14 이기 때문에, 8.13 이하 버젼은 8.14로 버젼업을 해야함.

PUT _inference/rerank/cohere_rerank <br>
{<br>
    "service": "cohere",<br>
    "service_settings": {<br>
        "api_key": <API-KEY>, <br>
        "model_id": "rerank-english-v3.0"<br>
    },<br>
    "task_settings": {<br>
        "top_n": 10,<br>
        "return_documents": true<br>
    }<br>
}

### 영어가 아니기 때문에 모델을 "rerank-multilingual-v3.0"로 바꿈.

## vectorstore : es8

# 필요한 패키지 설치

In [1]:
# !openai migrate
# openai 1.x.x 이상 버젼을 사용할 경우 위 명령을 수행해야 langchain_community오류가 해결됨. 
# 오류 메세지는 open.error.ApiError과 같은 오류.
# openai migrate가 동작하지 않을 경우 
# 아래 명령을 수행. grit이 설치 되어 있을 경우 1라인은 스킵 가능.
# curl -fsSL https://docs.grit.io/install | bash
# grit install
# grit apply openai

# 위 명령을 수행하면 python 의 site_package 에서 langchain_community에서 python파일을 로딩해서 오류 구문을 수정. 
# 단 아래와 같은 코드는 아직도 버그임. 
# | retry_if_exception_type(openai.ServiceUnavailableError)
# openai 에는 ServiceUnavailableError 에러 클래스가 없음..

In [2]:
import openai
print(openai.__version__)
import langchain_community
print(langchain_community.__version__)

1.37.1
0.2.10


In [3]:
# %pip install -qU openai
# %pip install -qU langchain
# %pip install langchainhub
# %pip install -qU langchain-openai
# %pip install -U langchain_community
# %pip install langchain_elasticsearch
# %pip install -U langchain-cohere
# %pip install -qU langchain-teddynote

In [4]:
import os
# from langchain_community.vectorstores.elastic_vector_search import ElasticKnnSearch
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy, ElasticsearchEmbeddings
from elasticsearch import Elasticsearch

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
# embedding 패키지 로딩
from langchain_community.embeddings import HuggingFaceHubEmbeddings
# from langchain_teddynote import logging

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ES_HOST = os.getenv("ES_HOST")
ES_USER_ID = os.getenv("ES_ID")
ES_PASSWD = os.getenv("ES_PWD")
ES_INDEX_NAME = os.getenv("ES_INDEX_NAME")
EMBEDDING_BASE_URL=os.getenv("EMBEDDING_BASE_URL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGUNGFACE_TOKEN")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

### embedding 정의

In [8]:
embeddings = HuggingFaceHubEmbeddings(model=EMBEDDING_BASE_URL, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpointEmbeddings`.
  warn_deprecated(


In [9]:
embeddings.client

<InferenceClient(model='http://192.168.0.10:3000', timeout=None)>

In [10]:
embeddings.embed_documents("테스트")[0][0:3]

[-0.017445877, 0.036139887, -0.017976798]

### llm. 정의

In [11]:
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

In [12]:
llm.invoke("안녕")

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'stop', 'logprobs': None}, id='run-d83245a5-5912-4a6d-a977-82b1f8255e01-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19})

### ES 모듈 로딩

In [13]:
es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)

/var/folders/jy/5_rl6575533638lskdr9s6s00000gn/T/ipykernel_28418/99351195.py:1: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://192.168.0.10:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [14]:
es.info()

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'name': 'es01', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'aZJrg1ffSsCl52WsqYpJvQ', 'version': {'number': '8.14.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd55f984299e0e88dee72ebd8255f7ff130859ad0', 'build_date': '2024-07-07T22:04:49.882652950Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Retriever 정의

In [15]:
knn_vector_store = ElasticsearchStore(
    ES_INDEX_NAME,
    embedding=embeddings,
    es_connection=es,
    strategy=DenseVectorStrategy(hybrid=True)
#     hybrid 서치를 위한 설정. 
#     해당 설정을 사용하기 위해서는 es8 라이센스가 상용이거나  trail이어야함.
)

knn_retriever = knn_vector_store.as_retriever()

### Cohere Reranker

In [16]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

In [17]:
# Cohere?

In [18]:
cohere_llm = Cohere(temperature=0, cohere_api_key=COHERE_API_KEY)
compressor = CohereRerank(model="rerank-multilingual-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=knn_retriever
)

compressed_docs = compression_retriever.invoke(
    "삼성전자"
)
print(compressed_docs)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[Document(metadata={'id': 209416, 'hash_key': '9c0236593da611e47d67a6b1cab1f958', 'title': "삼성전자, 또 52주 최고가 경신…'8만전자' 다가서", 'created_date': '2024-01-02T10:26:07', 'portal': 'daum', 'media': '한국경제', 'url': 'https://v.daum.net/v/20240102102607828', 'image_url': 'https://img1.daumcdn.net/thumb/S1200x630/?fname=https://t1.daumcdn.net/news/202401/02/ked/20240102102608958epwg.jpg', 'relevance_score': 0.9982926}, page_content='2일 오전 10시 20분 기준 삼성전자는 전 거래일 대비 300원(0.38%) 오른 7만8800원에 거래되고 있다. 장중 삼성전자는 7만9400원까지 오르며 52주 신고가를 기록했다. 전 거래일까지 삼성전자는 7거래일 연속 상승했다.\n이날 메리츠증권은 삼성전자에 대한 목표주가를 기존 9만4000원에서 9만5000원으로 높였다. 투자의견은 \'매수\'를 유지했다. 작년 4분기 삼성전자의 영업이익이 3분기에 비해 개선될 것으로 전망하면서다.\n김선우 메리츠증권 연구원은 "지난해 4분기 영업이익은 4조4000억원으로 전분기 2조4000억원 대비 소폭 개선될 것"이라며 "주로 메모리 매출 극대화에 집중하는 가운데 반도체(DS) 부문 영업손실이 전분기 3조7000억원에서 6000억원으로 감소하겠지만 모바일경험(MX) 부문 영업이익은 3조3000억원에서 2조3000억원으로 축소될 것"이라고 설명했다.\n이어 "삼성전자는 최근 예상보다 빠른 속도로 D램 감산폭을 줄이는 것으로 추정된다"며 "올해 2분기부터 감산폭 축소에서 오는 고정비 분배 및 수익성 회복이 예상된다"고 밝혔다.\n진영기 한경닷컴 기자 young71@hankyu

# KNN Search일 경우 query_vector를 같이 넘겨줘야 함. 

### rag prompt 로딩

In [20]:
# prompt = hub.pull("rlm/rag-prompt")
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

In [21]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [22]:
rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
# rag_chain.invoke("삼성전자 관련 기사를 검색하고 각각의 기사에 대한 내용을 요약하고 날짜를 같이 표시해줘")
# from langchain_teddynote.messages import stream_response
answer = rag_chain.invoke("삼성전자의 가장 최신 뉴스를 보여주세요")
print(answer)

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


삼성전자는 엔비디아에 4세대 고대역폭메모리(HBM)인 HBM3를 공급하기 시작했으며, 5세대 HBM3E는 아직 테스트 중에 있습니다. 이와 함께 최신 갤럭시S24 시리즈는 인공지능(AI) 기능을 탑재하여 아이폰15 시리즈보다 높은 만족도를 기록했습니다. 또한, 삼성전자는 최근 신제품 Neo QLED TV를 소개했습니다.


In [ ]:
# es.close()

### knn 검색일 경우 passthrough로 query로 query_vector가 넘어가지 않아서 오류 발생. 
### ES 8 같을 경우 huggingface 모델을 내장할수 있어서 model_id를 통해서 ES8 내부에서 처리 가능.